In [189]:
import os
import h5py
import cortex
import copy
import numpy as np
import matplotlib.pyplot as plt

## Setup results path

In [190]:
base_path = "/Users/mosorio/Documents/CAJAL-NeuroAI/Project/cajal_llm_project/results/"
experiment_name = "shuffled_words/percentage1.0"
experiment_baseline_name = "original_code"
results_path = os.path.join(base_path, experiment_name)
results_path
results_baseline_path = os.path.join(base_path, experiment_baseline_name)
#results_baseline_path

In [191]:
cortex.database.default_filestore

'/Users/mosorio/miniforge3/envs/llms/share/pycortex/db'

## Setup fMRI data from individual

In [192]:
surfaces = dict(
# F = 'fMRI_story_F',
# G = 'fMRI_story_G',
# H = 'fMRI_story_H',
I = 'fMRI_story_I',
# J = 'fMRI_story_J',
# K = 'fMRI_story_K',
# L = 'fMRI_story_L',
# M = 'fMRI_story_M',
# N = 'fMRI_story_N'
)

transforms = dict(
# F = 'F_ars',
# G = 'G_ars',
# H = 'H_ars',
I = 'I_ars',
# J = 'J_ars',
# K = 'K_ars',
# L = 'L_ars',
# M = 'M_ars',
# N = 'N_ars'
)

new_transforms = dict(
# F = 'F_ars_auto2',
# G = 'G_ars_auto2',
# H = 'H_ars_auto2',
I = 'I_ars_auto2',
# J = 'J_ars_auto2',
# K = 'K_ars_auto2',
# L = 'L_ars_auto2',
# M = 'M_ars_auto2',
# N = 'N_ars_auto2'
)

sub = 'I'

In [193]:
# check whether these transforms work for the PLoS data
mask = cortex.db.get_mask(surfaces[sub], transforms[sub], 'thin')
print('num voxels in transform for I: {}'.format(np.sum(mask)))

num voxels in transform for I: 25281


In [194]:
mask = cortex.db.get_mask(surfaces[sub], new_transforms[sub], 'thin')
print('num voxels in new_transform for I: {}'.format(np.sum(mask)))

num voxels in new_transform for I: 25263


In [195]:
# Open in read mode
with h5py.File(os.path.join(results_path, "results_encoding_model.h5"), 'r') as f:
    # Read metadata
    model_name = f.attrs['model_name']
    print("Model name:", model_name)

    # List all groups
    print("Groups available:", list(f.keys()))

    # Choose your group
    representation_name = list(f.keys())[0]  # or set explicitly
    group = f[representation_name]

    # Read datasets
    predictions = group['predictions'][:]
    ground_truth = group['ground_truth'][:]
    correlations = group['correlations'][:]
    p_values = group['p_values'][:]
    coefficients = group['coefficients'][:]
    alphas = group['alphas'][:]

    print("Predictions shape:", predictions.shape)

Model name: Llama-3.2-1B-Full-Chapter
Groups available: ['layer -7']
Predictions shape: (300, 25263)


In [196]:
# Open in read mode
with h5py.File(os.path.join(results_baseline_path, "results_encoding_model.h5"), 'r') as f:
    # Read metadata
    model_name_baseline = f.attrs['model_name']
    print("Model name:", model_name_baseline)

    # List all groups
    print("Groups available:", list(f.keys()))

    # Choose your group
    representation_name_baseline = list(f.keys())[0]  # or set explicitly
    group = f[representation_name_baseline]

    # Read datasets
    predictions_baseline = group['predictions'][:]
    ground_truth_baseline = group['ground_truth'][:]
    correlations_baseline = group['correlations'][:]
    p_values_baseline = group['p_values'][:]
    coefficients_baseline = group['coefficients'][:]
    alphas_baseline = group['alphas'][:]

    print("Predictions shape:", predictions_baseline.shape)

Model name: Llama-3.2-1B-Full-Chapter
Groups available: ['layer -7']
Predictions shape: (300, 25263)


In [197]:
dif_corr = correlations_baseline - correlations

In [198]:
correlations_baseline

array([ 0.0365065 ,  0.01593767, -0.03802222, ..., -0.06723929,
       -0.0146652 ,  0.041631  ], shape=(25263,), dtype=float32)

In [199]:
correlations

array([ 0.10836191,  0.07864682,  0.09338371, ..., -0.13676135,
       -0.18308775, -0.11579523], shape=(25263,), dtype=float32)

In [200]:
print(max(dif_corr))

0.6531676


In [201]:
# significant_voxels = np.where(p_values < 0.05, voxel_values_to_plot, 0.)
significant_voxels_baseline = copy.deepcopy(correlations_baseline)
significant_voxels_baseline[p_values > 0.05] = 0
significant_voxels_baseline.shape

(25263,)

In [202]:
# significant_voxels = np.where(p_values < 0.05, voxel_values_to_plot, 0.)
significant_voxels = copy.deepcopy(correlations)
significant_voxels[p_values > 0.05] = 0
significant_voxels.shape

(25263,)

## Plot all values

In [203]:
# vol = cortex.Volume(correlations, surfaces[sub], new_transforms[sub], vmin=0, vmax=0.6, cmap='viridis')
# cortex.webshow(vol)

## Plot significant values

In [204]:
vol = cortex.Volume(significant_voxels, surfaces[sub], new_transforms[sub], vmin=0, vmax=0.4, cmap='viridis')
cortex.webshow(vol)

Started server on port 60999
Stopping server


<JS: window.viewer>

In [205]:
surfaces[sub]

'fMRI_story_I'

In [206]:
# Create masks
mask_baseline = (significant_voxels_baseline != 0)
mask_current   = (significant_voxels != 0)
intersection   = mask_baseline & mask_current
only_baseline  = mask_baseline & ~mask_current
only_current   = mask_current & ~mask_baseline

In [207]:
np.any(only_current==True)

np.False_

In [208]:
only_baseline

array([False, False, False, ..., False, False, False], shape=(25263,))

In [209]:
sub

'I'

In [213]:
import numpy as np
import cortex
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Generate labeled volume
label_volume = np.zeros(significant_voxels.shape, dtype=np.int32)

only_current   = (significant_voxels != 0) & (significant_voxels_baseline == 0)
only_baseline  = (significant_voxels_baseline != 0) & (significant_voxels == 0)
intersection   = (significant_voxels != 0) & (significant_voxels_baseline != 0)

label_volume[only_current]   = 1
label_volume[only_baseline]  = 2
label_volume[intersection]   = 3

In [214]:
np.unique(label_volume)

array([0, 3], dtype=int32)

In [212]:
# Create cortex.Volume
vol = cortex.Volume(
    data=label_volume,
    subject=surfaces[sub],                     
    xfmname=new_transforms[sub],              
    vmin=0,
    vmax=1,
    cmap='Set1'
)

# Show in browser
cortex.webshow(vol)


Started server on port 52268
Stopping server


<JS: window.viewer>

In [57]:
n_total = len(correlations)
n_positive = np.sum(correlations > 0)
n_high = np.sum(correlations > 0.2)

print(f"{n_positive/n_total:.2%} of voxels have positive correlation")
print(f"{n_high/n_total:.2%} of voxels have corr > 0.2")


58.97% of voxels have positive correlation
0.18% of voxels have corr > 0.2


In [58]:
n_total_baseline = len(correlations_baseline)
n_positive_baseline = np.sum(correlations_baseline > 0)
n_high_baseline = np.sum(correlations_baseline > 0.2)

print(f"{n_positive_baseline/n_total_baseline:.2%} of voxels have positive correlation")
print(f"{n_high_baseline/n_total_baseline:.2%} of voxels have corr > 0.2")

87.63% of voxels have positive correlation
24.71% of voxels have corr > 0.2
